# Summary

Use [`python-fitbit`](http://python-fitbit.readthedocs.io/en/latest/#) to interact with the Fitbit API and download activity data. Convert data to pandas dataframe and write to disk.

In [1]:
import fitbit
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('poster')

In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
access_token = os.environ.get("ACCESS_TOKEN")
refresh_token = os.environ.get("REFRESH_TOKEN")
expires_at = float(os.environ.get("EXPIRES_AT"))

In [3]:
authd_client = fitbit.Fitbit(client_id, client_secret, access_token=access_token, 
                             refresh_token=refresh_token, expires_at=expires_at)

In [30]:
resources = [
    'activities/calories',  
    'activities/caloriesBMR',
    'activities/steps',
    'activities/distance',
    'activities/minutesSedentary',
    'activities/minutesLightlyActive',
    'activities/minutesFairlyActive',
    'activities/minutesVeryActive',
#     'activities/activityCalories'
]
df_activity = pd.DataFrame({})
for resource in resources:
    activity = resource.split('/')[1]
    resource_name = "-".join(resource.split('/'))
    activity_time_series = authd_client.time_series(resource, base_date='today', end_date='2016-11-09')
    df_activity[activity] = [resource_dict['value'] for resource_dict in activity_time_series[resource_name]]
df_activity['datetime'] = [resource_dict['dateTime'] for resource_dict in activity_time_series[resource_name]]

In [32]:
df_activity['datetime'] = pd.to_datetime(df_activity['datetime'])
df_activity = df_activity.set_index('datetime')

In [33]:
df_activity.head()

,calories,caloriesBMR,steps,distance,minutesSedentary,minutesLightlyActive,minutesFairlyActive,minutesVeryActive
datetime,,,,,,,,
2016-11-09,3200,1735,17760,8.47530422009856,712,301,29,68
2016-11-10,3733,1733,21888,10.47757346692784,449,322,98,106
2016-11-11,3094,1732,15847,7.69073609823168,572,284,66,51
2016-11-12,2994,1731,15215,7.26079694306304,620,273,61,31
2016-11-13,3009,1730,14414,6.91717867388704,616,307,32,49


In [34]:
data_path = os.path.join(os.getcwd(), os.pardir, 'data', 'interim', 'activity_data.csv')
df_activity.to_csv(data_path, index_label='datetime')